1. -gsp_persons.birth_year
2. -gsp_persons.birth_region_id
3. -gsp_persons.living_region_id
4. -doesLive_birth_region ( 0,1 )
4. -gsp_persons.nationality_id
5. -gsp_persons.sex_id
6. -mib_orders 3 types -> ( jarayonda, tugallangan, yo'q )
7. -work 3 types -> yaxshi, o'rta, yomon
8. -zags 3 types -> ( turmush qurgan, turmush qurmagan, ajrashgan )
9. -ma'lumoti ( oliy, o'rta, unknown )

In [1]:
import pandas as pd
import numpy as np
import clickhouse_connect
import bisect
import random
from typing import List, Tuple, Optional

In [38]:
def get_pinpp_of_defendant_with_limit(limit=800):
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer" 
    password = ""
    query = f"""
SELECT pinpp  FROM defendants
WHERE toDate(qamalgan_sana) >= '2022-03-31'
ORDER BY qamalgan_sana DESC
limit {limit}
    """
    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        user=user,
        password=password,
        secure=True
    )
    query = query
    result = client.query(query)
    client.close()
    

    return result.result_rows

# result=get_pinpp_of_defendant_with_limit()

# df_def=pd.DataFrame(result, columns=['pinfl'])
# df_def['jinoyatchimi']=1

# pinfl_list_to_check = list(df_def['pinfl'])

In [39]:
def get_all_pinpp_of_defendant():
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer"
    password = ""

    query = """SELECT pinpp FROM defendants"""
    
    client = clickhouse_connect.get_client(host=host, port=port, user=user, password=password)
    result = client.query(query)
    client.close()

    return result.result_rows

all_defendants_pinpp -> list 

limited_defendants_pinpp -> all_defendants_pinpp[:1000]



In [40]:
def get_pinpp_of_normal_humans(not_get_pinfl_list, limit=100000):
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer" 
    password = ""
    excluded_pinpps = ", ".join([f"'{pinpp}'" for pinpp in not_get_pinfl_list])
    
    query = f"""
    SELECT OriginAirportID
    FROM ontime
    WHERE OriginAirportID NOT IN ({excluded_pinpps})
    LIMIT {limit}
    """

    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        user=user,
        password=password,
        secure=True 
    )
    
    result = client.query(query)
    client.close() 
    return result.result_rows

# result=get_pinpp_of_normal_humans(pinfl_list_to_check, limit=100000)

# df_normal=pd.DataFrame(result, columns=['pinfl'])
# df_normal['jinoyatchimi']=0



In [ ]:
#### Bu ikki funksiya birinchi listdagi va ikkinchi listda yo'q ma'lumotlarni qaytaradi
#for pinpp -> list1 = limited normal people pinpp, list2 = all defendants pinpp

def check_isUnique(list1, list2):
    return [item for item in list1 if item not in list2]


def check_isUnique_with_bisect(list1, list2):
    list2.sort()
    result = []
    for item in list1:
        pos = bisect.bisect_left(list2, item)
        if pos == len(list2) or list2[pos] != item:
            result.append(item)

    return result

list1 = list(range(1, 100_000))

list2 = list(range(80_000, 120_000))

result = check_isUnique(list1, list2)
print(len(result))

79999


In [46]:
import time

start_time = time.time()
check_isUnique(list1, list2)
end_time = time.time()
print(f"1. {end_time - start_time}")

start_time = time.time()
check_isUnique_with_bisect(list1, list2)
end_time = time.time()
print(f"2. {end_time - start_time}.")

1. 24.12644672393799
2. 0.02007746696472168.


In [42]:
def get_data_by_pinpp(
    pinpp_list_overall,
    chunk_size,
    limit):
        config = {
            "host": "play.clickhouse.com",
            "port": 443,
            "user": "explorer",
            "password": "",
            "secure": True
        }
        
        with clickhouse_connect.get_client(**config) as client:
            result = []
            
            chunks = [
                pinpp_list_overall[i:i + chunk_size] 
                for i in range(0, len(pinpp_list_overall), chunk_size)
            ]
            
            for chunk in chunks:
                query = """
                SELECT Year, Reporting_Airline, Month
                FROM ontime
                WHERE OriginAirportID IN ({})
                """
                
                if limit:
                    query += f" LIMIT {limit}"
                
                placeholders = ", ".join([f"'{pinpp}'" for pinpp in chunk])
                formatted_query = query.format(placeholders)
                
                partial_result = client.query(formatted_query)
                result.extend(partial_result.result_rows)
            
            return result
        
# pinpp_list = pinfl_list_to_check
# results = get_data_by_pinpp(
#     pinpp_list_overall=pinpp_list,
#     chunk_size=50,
#     limit=500
# )

# df_overall_data = pd.DataFrame(results, columns=['Year', 'Reporting_Airline', 'Month'])
# df_overall_data.sort_values(by='Year', ascending=True)